## Здесь извлеченная информация превращается в презентацию.

In [ ]:
if __name__ == '__main__' and '__file__' not in globals():
    %reset

In [ ]:
if __name__ == '__main__' and '__file__' not in globals():
    %run common_info.ipynb

In [ ]:
%run process_pdf.ipynb

In [ ]:
from pprint import pprint

pprint(presentationContent)

### Заготавливаем модель презентации.

In [ ]:
from collections import Counter
from copy import deepcopy

slideHeaders = Counter()
presentationModel = []
# из-за особенностей построения презентации
involvedImages = []
involvedImageReferences = set()
involvedTables = []
involvedTableReferences = set()
involvedSentences = set()
for content in presentationContent:
    # делим наполнение между слайдами
    # к текстовой части можно вставить одно изображение
    # если в предложении есть ссылка на изображение и предложение - не часть списка, 
    # то изображение и это предложение можно вытащить в отдельный слайд
    # если в предложении есть ссылка на изображение и предложение - часть списка, 
    # то изображение остается на слайде (если место еще не занято и изображение с такой подписью - единственное)
    # если в предложении есть ссылка на единственную таблицу и оно - не часть списка, то вытаскиваем предложение на слайд с таблицей
    # если в секции есть таблица, то выносим ее в отдельный слайд
    # если в секции есть изображение, то выносим его в отдельный слайд
    # если на слайде только одно изображение, то его подпись уходит в заголовок
    header = content['bareHeader'] if content['bareHeader'].lower() != 'введение' else 'Актуальность задачи'
    additionToHeader = ' (' + str(slideHeaders[header]) + ')' if header in slideHeaders else ''
    slides = [{
        'header': header + additionToHeader,
        'content': [],
        'image': {}
        
    }]
    slideHeaders[header] += 1
    
    # сначала добавляем слайды с таблицами (и удаляем предложения из общего списка, если они хранят ссылки на таблицы)
    sentences = [] # предложения, которые остались после извлечения таблиц на отдельные слайды
    for sentence in content['importantSentences']:
        tables = [table for table in captionsToTables if table['numeration'] in content['tableReferences']]
        # удаляем таблицы, которые уже были добавлены в слайды
        tables = [table for table in tables if table not in involvedTables]
        tableReferences = set([reference for reference in content['tableReferences'] if reference not in involvedTableReferences])
        # потому что далее эти объекты меняются
        sentenceCopy = deepcopy(sentence)
        # каждая таблица просто получает свой слайд, с предложением или без
        if not sentence['bulletListPart'] and not sentence['numberedListPart'] and \
           sentence['tableReferences'].intersection(tableReferences) and \
           not sentence['imageReferences']:
            for table in tables:
                if sentence['sentence'] not in involvedSentences:
                    slides.append({
                        'header': header + ' (' + str(slideHeaders[header]) + ')',
                        'content': [sentenceCopy],
                        'image': table
                    })
                    involvedSentences.add(sentenceCopy['sentence'])
                else:
                    slides.append({
                        'header': header + ' (' + str(slideHeaders[header]) + ')',
                        'content': [],
                        'image': table
                    })
                slideHeaders[header] += 1
                involvedTables.append(table)
                involvedTableReferences = involvedTableReferences.union(set(
                    [table['numeration'] for table in tables]
                ))
        else:
            for table in tables:
                slides.append({
                    'header': header + ' (' + str(slideHeaders[header]) + ')',
                    'content': [],
                    'image': [table]
                })
                slideHeaders[header] += 1
                involvedTables.append(table)
                involvedTableReferences = involvedTableReferences.union(set(
                    [table['numeration'] for table in tables]
                ))
            sentences.append(sentenceCopy)
        
    # добавляем слайды с таблицами, которые относятся к разделу и еще не были добавлены
    tableReferences = content['tableReferences'].difference(involvedTableReferences)
    tables = [table for table in captionsToTables if table['numeration'] in tableReferences]
    for table in tables:
        slides.append({
            'header': header + '(' + str(slideHeaders[header]) + ')',
            'content': [],
            'image': [table]
        })
        slideHeaders[header] += 1
        involvedTables.append(table)
        involvedTableReferences = involvedTableReferences.union(set(
            [table['numeration'] for table in tables]
        ))
                
    # добавляем слайды с изображениями
    for sentence in sentences:
        images = [image for image in captionsToImages if image['numeration'] in sentence['imageReferences']]
        # удаляем изображения, которые уже были добавлены в слайды
        images = [image for image in images if image not in involvedImages]
        imageReferences = set([reference for reference in sentence['imageReferences'] if reference not in involvedImageReferences])
        # потому что далее эти объекты меняются
        sentenceCopy = deepcopy(sentence)
        # просто предложение без ссылок на изображения
        if not imageReferences:
            if sentence['sentence'] not in involvedSentences:
                slides[0]['content'].append(sentenceCopy)
                involvedSentences.add(sentenceCopy['sentence'])
        # предложение со ссылкой на единственное изображение (и других изображений с такой подписью нет)
        elif not slides[0]['image'] and \
             (len(imageReferences) == 1 and len(images) == 1) and \
             (images[0]['height'] / images[0]['width'] >= 1.5 or images[0]['height'] / images[0]['width'] <= 0.3):
            if sentence['sentence'] not in involvedSentences:
                slides[0]['content'].append(sentenceCopy)
                involvedSentences.add(sentenceCopy['sentence'])
            slides[0]['image'] = images[0]
            involvedImages.append(images[0])
            involvedImageReferences.add(images[0]['numeration'])                
        # предложение со ссылкой на единственное изображение и место на первом слайде уже занято или избражение не подходт по габаритам
        elif (len(imageReferences) == 1 and len(images) == 1) and \
             (slides[0]['image'] or \
              (images[0]['height'] / images[0]['width'] < 1.5 and images[0]['height'] / images[0]['width'] > 0.3)):
            if not sentence['bulletListPart'] and not sentence['numberedListPart']:
                # можно утащить предложение на выделенный слайд
                if sentence['sentence'] not in involvedSentences:
                    slides.append({
                        'header': header + ' (' + str(slideHeaders[header]) + ')',
                        'content': [sentenceCopy],
                        'image': images[0]
                    })
                    involvedSentences.add(sentenceCopy['sentence'])
                else:
                    slides.append({
                        'header': header + ' (' + str(slideHeaders[header]) + ')',
                        'content': [],
                        'image': images[0]
                    })
            else:
                if sentence['sentence'] not in involvedSentences:
                    slides[0]['content'].append(sentenceCopy)
                    involvedSentences.add(sentenceCopy['sentence'])
                # изображение просто получает свой слайд
                slides.append({
                    'header': header + ' (' + str(slideHeaders[header]) + ')',
                    'content': [],
                    'image': [images[0]]
                })
            slideHeaders[header] += 1
            involvedImages += images
            involvedImageReferences = involvedImageReferences.union(set(
                [image['numeration'] for image in images]
            ))
        # предложение со ссылкой на единственное изображение (но есть другие изображения с такой подписью)
        elif len(imageReferences) == 1 and len(images) > 1:
            if sentence['sentence'] not in involvedSentences:
                slides[0]['content'].append(sentenceCopy)
                involvedSentences.add(sentenceCopy['sentence'])
            slides.append({
                'header': header + ' (' + str(slideHeaders[header]) + ')',
                'content': [],
                'image': images
            })
            slideHeaders[header] += 1
            involvedImages += images
            involvedImageReferences = involvedImageReferences.union(set(
                [image['numeration'] for image in images]
            ))
        # предложение со ссылкой на несколько изображений (у каждого из которых своя подпись)
        elif len(imageReferences) > 1 and len(images) == len(imageReferences):
            if sentence['sentence'] not in involvedSentences:
                slides[0]['content'].append(sentenceCopy)
                involvedSentences.add(sentenceCopy['sentence'])
            if not slides[0]['image'] and \
               (images[0]['height'] / images[0]['width'] > 1.5 and images[0]['height'] / images[0]['width'] < 0.3):
                slides[0]['image'] = images[0]
                involvedImages.append(images[0])
                involvedImageReferences.add(images[0]['numeration'])
                images = images[1:]
            for image in images:
                slides.append({
                    'header':header + ' (' + str(slideHeaders[header]) + ')',
                    'content': [],
                    'image': [image]
                })
                slideHeaders[header] += 1
                involvedImages += images
                involvedImageReferences = involvedImageReferences.union(set(
                    [image['numeration'] for image in images]
                ))
        # предложение со ссылкой на несколько изображений (у некоторых из которых может быть одна подпись)
        elif len(imageReferences) > 1 and len(images) != len(imageReferences):
            if sentence['sentence'] not in involvedSentences:
                slides[0]['content'].append(sentenceCopy)
                involvedSentences.add(sentenceCopy['sentence'])
            for imageReference in sentence['imageReferences']:
                sameCaptionImages = [image for image in images if image['numeration'] == imageReference]
                if not slides[0]['image'] and len(sameCaptionImages) == 1:
                    slides[0]['image'] = sameCaptionImages[0]
                    involvedImages.append(sameCaptionImages[0])
                    involvedImageReferences.add(sameCaptionImages[0]['numeration'])
                else:
                    slides.append({
                        'header': header + ' (' + str(slideHeaders[header]) + ')',
                        'content': [],
                        'image': images 
                    })
                    slideHeaders[header] += 1
                    involvedImages += images
                    involvedImageReferences = involvedImageReferences.union(set(
                        [image['numeration'] for image in images]
                    ))
                    
    # добавляем слайд с изображениями, которые относятся к разделу и еще не были добавлены
    images = [image for image in captionsToImages if image['numeration'] in content['sectionImageReferences']]
    if images:
        slides.append({
            'header': header + ' (' + str(slideHeaders[header]) + ')',
            'content': [],
            'image': images
        })
        slideHeaders[header] += 1
        involvedImages += images
        involvedImageReferences = involvedImageReferences.union(set(
            [image['numeration'] for image in images]
        ))
    
    if (slides[0]['content'] and len(slides[0]['content']) > 1) or \
        slides[0]['image']:
        presentationModel += slides
    else:
        presentationModel += slides[1:]
    
pprint(presentationModel)

### Собираем титульный слайд.

In [ ]:
from pptx.util import Pt, Inches
from pptx.enum.text import PP_ALIGN

def addTitleSlide(titlePageDict, presentation):
    titleSlideLayout = presentation.slide_layouts[0]
    titleSlide = presentation.slides.add_slide(titleSlideLayout)
    title = titleSlide.shapes.title

    # заголовок научной работы
    titleParagraph = title.text_frame.paragraphs[0]
    titleParagraph.line_spacing = Pt(1)
    titleParagraph.alignment = PP_ALIGN.CENTER
    titleRun = titleParagraph.add_run()
    titleRun.text = titlePageDict['workTitle']
    titleFont = titleRun.font
    titleFont.name = 'Arial Black'
    titleFont.size = Pt(28)

    signatureTextbox = titleSlide.shapes.add_textbox(Inches(5), Inches(5), Inches(5), Inches(5))

    # кто подготовил презентцию
    signatureTextboxAuthorParagraph = signatureTextbox.text_frame.paragraphs[0]
    signatureTextboxAuthorParagraph.alignment = PP_ALIGN.RIGHT
    signatureTextboxAuthorParagraph.space_after = Pt(1)
    signatureTextboxAuthorRun = signatureTextboxAuthorParagraph.add_run()
    signatureTextboxAuthorRun.text = "Подготовил(-a):"
    signatureTextboxAuthorFont = signatureTextboxAuthorRun.font
    signatureTextboxAuthorFont.name = 'Arial'
    signatureTextboxAuthorFont.size = Pt(20)
    signatureTextboxAuthorFont.underline = True

    signatureTextbox.text_frame.add_paragraph()
    signatureTextboxAuthorParagraph = signatureTextbox.text_frame.paragraphs[1]
    signatureTextboxAuthorParagraph.alignment = PP_ALIGN.RIGHT
    signatureTextboxAuthorParagraph.space_after = Pt(10)
    signatureTextboxAuthorRun = signatureTextboxAuthorParagraph.add_run()
    signatureTextboxAuthorRun.text = titlePageDict['nameAndSurname']
    signatureTextboxAuthorFont = signatureTextboxAuthorRun.font
    signatureTextboxAuthorFont.name = 'Arial'
    signatureTextboxAuthorFont.size = Pt(20)

    # научный руководитель
    signatureTextbox.text_frame.add_paragraph()
    signatureTextboxScitificDirectorParagraph = signatureTextbox.text_frame.paragraphs[2]
    signatureTextboxScitificDirectorParagraph.alignment = PP_ALIGN.RIGHT
    signatureTextboxScitificDirectorParagraph.space_after = Pt(1)
    signatureTextboxScitificDirectorRun = signatureTextboxScitificDirectorParagraph.add_run()
    signatureTextboxScitificDirectorRun.text = "Научный руководитель:"
    signatureTextboxScitificDirectorFont = signatureTextboxScitificDirectorRun.font
    signatureTextboxScitificDirectorFont.name = 'Arial'
    signatureTextboxScitificDirectorFont.size = Pt(20)
    signatureTextboxScitificDirectorFont.underline = True

    signatureTextbox.text_frame.add_paragraph()
    signatureTextboxScitificDirectorParagraph = signatureTextbox.text_frame.paragraphs[3]
    signatureTextboxScitificDirectorParagraph.alignment = PP_ALIGN.RIGHT
    signatureTextboxScitificDirectorParagraph.line_spacing = Pt(1)
    signatureTextboxScitificDirectorRun = signatureTextboxScitificDirectorParagraph.add_run()
    signatureTextboxScitificDirectorRun.text = titlePageDict['scitificDirectorPosition'] + '\n' + \
                                               titlePageDict['scitificDirectorNameAndSurname']
    signatureTextboxScitificDirectorFont = signatureTextboxScitificDirectorRun.font
    signatureTextboxScitificDirectorFont.name = 'Arial'
    signatureTextboxScitificDirectorFont.size = Pt(20)

    # удаление пустых плейсхолдеров
    for placeholder in titleSlide.shapes.placeholders:
        if placeholder.has_text_frame and placeholder.text_frame.text == "":
            sp = titleSlide.placeholders[1]._sp
            sp.getparent().remove(sp)

### Собираем слайд только с текстом.

In [ ]:
from math import ceil

def preprocessText(content):
    # содержит ли наполнение слайда обобщающее предложение для элементов маркерного списка
    # если такого предложения нет, то делаем каждое предложение элементом маркерного списка на слайде
    # если такое предложение есть, то элементами маркерного списка становятся 
    # только оригинальные элемены маркерного списка и предложения, которые были между ними
    containsBulletList = 0
    containsBulletListContent = 0
    numbering = 1
    for sentence in content:
#         print(sentence['sentence'])
        if sentence['sentence'].split(' ', 1)[0][0] == '0':
            sentence['numberedListPart'] = False # у таких предложений не должно быть номера
        if sentence['sentence'].split(' ', 1)[0][0] in numbers: # обрезаем номера из начала
            sentence['sentence'] = sentence['sentence'].split(' ', 1)[1].strip()
#         print(sentence['sentence'].split(' ')[-1][0])
        if sentence['sentence'].split(' ')[-1][-1] in numbers: # обрезаем номера с конца
            sentence['sentence'] = ' '.join(sentence['sentence'].split(' ')[:-1]).strip()
#         print(sentence['sentence'].split(' ', 1)[0][0])
        if sentence['sentence'].split(' ', 1)[0][0] in listMarkers: # обрезаем маркер списка, чтобы не мешался
            sentence['sentence'] = sentence['sentence'].split(' ', 1)[1].strip()
#         print(sentence['sentence'], sentence['bulletListPart'], sentence['sentence'][-1])
#         print(sentence['sentence'])
    
        if sentence['bulletListPart'] and sentence['sentence'][-1] == ':':
            sentence['bulletListPart'] = False
            sentence['numberedListPart'] = False
            containsBulletList = True
        if containsBulletList and (sentence['numberedListPart'] or sentence['betweenNumbering']):
            sentence['numberedListPart'] = True # у таких предложений должен быть номер
            # добавляем свой номер
            sentence['sentence'] = str(numbering) + '. ' + sentence['sentence']
            numbering += 1
            containsBulletListContent += 1
        if containsBulletList and (sentence['bulletListPart'] or sentence['betweenBullets']):
            sentence['bulletListPart'] = True # у таких предложений должен быть маркер списка
            containsBulletListContent += 1
        if not containsBulletList and (sentence['bulletListPart'] or sentence['numberedListPart']):
            sentence['bulletListPart'] = False # у таких предложений не должно быть маркера списка
            sentence['numberedListPart'] = False # у таких предложений не должно быть номера

    if (not containsBulletList or (containsBulletList and containsBulletListContent < 2)) and len(content) > 1:
        for sentence in content:
            if sentence['sentence'][-1] == ':':
                sentence['sentence'] = sentence['sentence'][0:-1] + '.'
            if sentence['sentence'][-1] not in punctuation:
                sentence['sentence'] = sentence['sentence'] + '.'
            if sentence['sentence'].split(' ', 1)[0][0] in numbers: # обрезаем номера из начала
                sentence['sentence'] = sentence['sentence'].split(' ', 1)[1].strip()
                sentence['numberedListPart'] = False
            sentence['bulletListPart'] = True # у таких предложений должен быть маркер списка

    pprint(content)
#     print(containsBulletList, containsBulletListContent)

In [ ]:
from pptx.util import Pt, Inches
from pptx.enum.text import PP_ALIGN
from lxml import etree

def addTextSlide(header, content, presentation, pageNumber):
#     print(content)
    # текстовый слайд
    textSlideLayout = presentation.slide_layouts[1]
    textSlide = presentation.slides.add_slide(textSlideLayout)
    title = textSlide.shapes.title
    body = textSlide.shapes.placeholders[1]
    
    # двигаем текст повыше
    bodyLeft = body.left
    bodyTop = body.top - Inches(0.3)
    bodyWidth = body.width
    bodyHeight = body.height
    body.left, body.top, body.width, body.height = bodyLeft, bodyTop, bodyWidth, bodyHeight
    
    # заголовок текстового слайда
    titleParagraph = title.text_frame.paragraphs[0]
    titleParagraph.line_spacing = Pt(1)
    titleParagraph.alignment = PP_ALIGN.LEFT
    titleRun = titleParagraph.add_run()
    titleRun.text = header
    titleFont = titleRun.font
    titleFont.name = 'Arial Black'
    titleFont.size = Pt(28)
    
    # содержимое текстового слайда
    for sentenceIndex in range(0, len(content)-1):
        body.text_frame.add_paragraph()
        
    preprocessText(content)
    
    lineSpace = 0
    for sentenceIndex in range(0, len(content)):
        lineSpace += ceil(len(content[sentenceIndex]['sentence']) / 70) * Inches(0.35) # сколько примерно места займет текст
        if lineSpace > slideHeight - bodyTop:
            break
        sentenceParagraph = body.text_frame.paragraphs[sentenceIndex]
        sentenceParagraph.alignment = PP_ALIGN.LEFT
        sentenceParagraph.line_spacing = Pt(1)
        sentenceParagraph.space_after = Pt(5)
        sentenceParagraphRun = sentenceParagraph.add_run()
        sentenceParagraphFont = sentenceParagraphRun.font
        sentenceParagraphFont.name = 'Arial'
        sentenceParagraphFont.size = Pt(18)
        sentenceParagraphRun.text = content[sentenceIndex]['sentence']
        if not content[sentenceIndex]['bulletListPart'] or content[sentenceIndex]['numberedListPart']:
            sentenceParagraph._pPr.insert(0, etree.Element("{http://schemas.openxmlformats.org/drawingml/2006/main}buNone"))
        if content[sentenceIndex]['bulletListPart']:
            sentenceParagraph.level = 0
            
    # добавляем нумерацию слайда
    pageNumberTextbox = textSlide.shapes.add_textbox(slideWidth - Inches(0.5), slideHeight - Inches(0.5), Inches(0.5), Inches(0.5))
    pageNumberParagraph = pageNumberTextbox.text_frame.paragraphs[0]
    pageNumberParagraph.alignment = PP_ALIGN.CENTER
    pageNumberRun = pageNumberParagraph.add_run()
    pageNumberRun.text = pageNumber
    pageNumberFont = pageNumberRun.font
    pageNumberFont.name = 'Arial'
    pageNumberFont.size = Pt(15)

    # удаление пустых плейсхолдеров
    for placeholder in textSlide.shapes.placeholders:
        if placeholder.has_text_frame and placeholder.text_frame.text == "":
            sp = textSlide.placeholders[1]._sp
            sp.getparent().remove(sp)

### Собираем слайд только с изображенияем (или несколькими изображениями).

In [ ]:
from pptx.util import Pt, Inches
from pptx.enum.text import PP_ALIGN

def addImageSlide(header, images, presentation, pageNumber):
    # слайд с текстом и изображением
    imageSlideLayout = presentation.slide_layouts[1]
    imageSlide = presentation.slides.add_slide(imageSlideLayout)
    title = imageSlide.shapes.title
    body = imageSlide.shapes.placeholders[1]
    
    # двигаем текст повыше
    bodyLeft = body.left
    bodyTop = body.top - Inches(0.3)
    bodyWidth = body.width
    bodyHeight = body.height
    body.left, body.top, body.width, body.height = bodyLeft, bodyTop, bodyWidth, bodyHeight
    
    # заголовок текстового слайда
    titleParagraph = title.text_frame.paragraphs[0]
    titleParagraph.line_spacing = Pt(1)
    titleParagraph.alignment = PP_ALIGN.LEFT
    titleRun = titleParagraph.add_run()
    titleRun.text = header
    titleFont = titleRun.font
    titleFont.name = 'Arial Black'
    titleFont.size = Pt(28)    
            
    # вставляем изображения в слайд
    # если изображение единственное, то просто помещаем его в центр слайда
    if len(images) == 1:
        heightToWidthRatio = images[0]['height'] / images[0]['width']
        top = body.top
        height = slideHeight - body.top - Inches(0.7)
        width = int(height / heightToWidthRatio)
        left = int((slideWidth - width) / 2)
        if (slideWidth - width) < Inches(1):
            width = int(slideWidth - Inches(1))
            height = int(width * heightToWidthRatio)
            left = int((slideWidth - width) / 2)
            top = body.top + int((slideHeight - body.top - height - Inches(0.7)) / 2)
        image = imageSlide.shapes.add_picture(images[0]['filePath'], Inches(0.5), top, height=height)
        totalWidth = width
        captionTop = top + height + Inches(0.2)
    # если нужно вставить больше одного предложения в слайд
    # будем выравнивать изображения по нижнему краю
    else:
        left = Inches(0.5)
        top = body.top
        for image in images:
            heightToWidthRatio = image['height'] / image['width']
            totalWidth = 0
            slideImages = []
            # проверяем, насколько изображение близко к квадрату
            if heightToWidthRatio >= 1: # подбираем размер по высоте
                height = slideHeight - body.top - Inches(0.7)
                width = int(height / heightToWidthRatio)
                if width <= (slideWidth - Inches(1)) / len(images):
                    image = imageSlide.shapes.add_picture(images[0]['filePath'], left, top, height=height)
                else: # подбираем размер по ширине, если при подборе по высоте она слишком большая
                    width = (slideWidth - Inches(1)) / len(images)
                    image = imageSlide.shapes.add_picture(images[0]['filePath'], left, top, width=width)
                    imageLeft = image.left
                    imageTop = top + (slideHeight - Inches(0.7) - image.height)
                    imageWidth = image.width
                    imageHeight = image.height
                    image.left, image.top, image.width, image.height = imageLeft, imageTop, imageWidth, imageHeight
            else: # подбираем размер по ширине
                width = (slideWidth - Inches(1)) / len(images)
                image = imageSlide.shapes.add_picture(images[0]['filePath'], left, top, width=width)
                imageLeft = image.left
                imageTop = top + (slideHeight - Inches(0.7) - image.height)
                imageWidth = image.width
                imageHeight = image.height
                image.left, image.top, image.width, image.height = imageLeft, imageTop, imageWidth, imageHeight
            left = left + image.left + Inches(0.5)
            totalWidth += image.width
            slideImages.append(image)
            
        left = int((slideWidth - totalWidth) / 2)
        for image in slideImages:
            imageLeft = left
            imageTop = image.top
            imageWidth = image.width
            imageHeight = image.height
            image.left, image.top, image.width, image.height = imageLeft, imageTop, imageWidth, imageHeight
            left += image.left + Inches(0.5)
        left = int((slideWidth - totalWidth) / 2)
        captionTop = slideHeight - Inches(0.7) + Inches(0.2)

    # вставляем подпись к изображению (или изображениям) в слайд
    imageCaption = imageSlide.shapes.add_textbox(left, captionTop, width=totalWidth, height=Inches(2))
    imageCaptionParagraph = imageCaption.text_frame.paragraphs[0]
    imageCaptionParagraph.alignment = PP_ALIGN.CENTER
    imageCaptionParagraph.line_spacing = Pt(1)
    imageCaptionRun = imageCaptionParagraph.add_run()
    imageCaptionRun.text = images[0]['caption']
    imageCaptionFont = imageCaptionRun.font
    imageCaptionFont.name = 'Arial'
    imageCaptionFont.size = Pt(15)
    
    # добавляем нумерацию слайда
    pageNumberTextbox = imageSlide.shapes.add_textbox(slideWidth - Inches(0.5), slideHeight - Inches(0.5), Inches(0.5), Inches(0.5))
    pageNumberParagraph = pageNumberTextbox.text_frame.paragraphs[0]
    pageNumberParagraph.alignment = PP_ALIGN.CENTER
    pageNumberRun = pageNumberParagraph.add_run()
    pageNumberRun.text = pageNumber
    pageNumberFont = pageNumberRun.font
    pageNumberFont.name = 'Arial'
    pageNumberFont.size = Pt(15)

    # удаление пустых плейсхолдеров
    for placeholder in imageSlide.shapes.placeholders:
        if placeholder.has_text_frame and placeholder.text_frame.text == "":
            sp = imageSlide.placeholders[1]._sp
            sp.getparent().remove(sp)

### Собираем слайд с текстом и одним изображением.

In [ ]:
from pptx.util import Pt, Inches
from pptx.enum.text import PP_ALIGN

def addTextAndImageSlide(header, content, image, presentation, pageNumber):
    # слайд с текстом и изображением
    textAndImageSlideLayout = presentation.slide_layouts[1]
    textAndImageSlide = presentation.slides.add_slide(textAndImageSlideLayout)
    title = textAndImageSlide.shapes.title
    body = textAndImageSlide.shapes.placeholders[1]
    
    # двигаем текст повыше
    bodyLeft = body.left
    bodyTop = body.top - Inches(0.3)
    bodyWidth = body.width
    bodyHeight = body.height
    body.left, body.top, body.width, body.height = bodyLeft, bodyTop, bodyWidth, bodyHeight
    
    # заголовок текстового слайда
    titleParagraph = title.text_frame.paragraphs[0]
    titleParagraph.line_spacing = Pt(1)
    titleParagraph.alignment = PP_ALIGN.LEFT
    titleRun = titleParagraph.add_run()
    titleRun.text = header
    titleFont = titleRun.font
    titleFont.name = 'Arial Black'
    titleFont.size = Pt(28)
    
    # содержимое текстового блока
    for sentenceIndex in range(0, len(content)-1):
        body.text_frame.add_paragraph()
        
    preprocessText(content)
    
    lineSpace = 0
    for sentenceIndex in range(0, len(content)):
        lineSpace += ceil(len(content[sentenceIndex]['sentence']) / 70) * Inches(0.35) # сколько примерно места займет текст
        sentenceParagraph = body.text_frame.paragraphs[sentenceIndex]
        sentenceParagraph.alignment = PP_ALIGN.LEFT
        sentenceParagraph.line_spacing = Pt(1)
        sentenceParagraph.space_after = Pt(5)
        sentenceParagraphRun = sentenceParagraph.add_run()
        sentenceParagraphFont = sentenceParagraphRun.font
        sentenceParagraphFont.name = 'Arial'
        sentenceParagraphFont.size = Pt(18)
        sentenceParagraphRun.text = content[sentenceIndex]['sentence']
        if not content[sentenceIndex]['bulletListPart'] or content[sentenceIndex]['numberedListPart']:
            sentenceParagraph._pPr.insert(0, etree.Element("{http://schemas.openxmlformats.org/drawingml/2006/main}buNone"))
        if content[sentenceIndex]['bulletListPart']:
            sentenceParagraph.level = 0
            
        # вставляем изображение в слайд
    heightToWidthRatio = image['height'] / image['width']
#     print(heightToWidthRatio)
    # изображение будет справа
    if heightToWidthRatio > 1.5:
        left = Inches(5)
        top = body.top
        width = slideWidth - left - Inches(0.3) - Inches(0.5)
        height = int(heightToWidthRatio * width)
        if (slideHeight - top - height) < Inches(0.7):
            height = int(slideHeight - top - Inches(0.7))
            width =  int(height / heightToWidthRatio)
            left = left + int((slideWidth - left - width) / 2)
        # это нужно, чтобы сделать текстовый блок нормального размера
        bodyLeft = body.left
        bodyTop = body.top
        bodyWidth = left - body.left - Inches(0.3)
        bodyHeight = body.height
        body.left, body.top, body.width, body.height = bodyLeft, bodyTop, bodyWidth, bodyHeight
        # добавляем изображение
        slideImage = textAndImageSlide.shapes.add_picture(image['filePath'], left, top, width=width)
         # это нужно для того, чтобы потом вставить подпись в правильное место
        captionTop = slideImage.top + slideImage.height + Inches(0.2)
        captionLeft = left + Inches(0.3)
        captionWidth = slideImage.width
    # изображение будет внизу
    if heightToWidthRatio < 0.3:
        left = Inches(0.5)
        top = body.top + lineSpace
        width = slideWidth - Inches(0.5) - Inches(0.5)
        height = int(heightToWidthRatio * width)
        if (top + height + Inches(0.7)) > slideHeight:
            height = int(slideHeight - top - Inches(0.7))
            width =  int(height / heightToWidthRatio)
            left = int((slideWidth - width) / 2)
        # это нужно, чтобы сделать текстовый блок нормального размера
        bodyLeft = body.left
        bodyTop = body.top
        bodyWidth = body.width
        bodyHeight = top - body.top - Inches(0.2)
        body.left, body.top, body.width, body.height = bodyLeft, bodyTop, bodyWidth, bodyHeight
        # добавляем изображение
        slideImage = textAndImageSlide.shapes.add_picture(image['filePath'], left, top, width=width)
        # это нужно для того, чтобы потом вставить подпись в правильное место
        captionTop = top + heightToWidthRatio * width + Inches(0.2)
        captionLeft = left
        captionWidth = slideImage.width
        
    # вставляем подпись к изображению в слайд
#     print(heightToWidthRatio, image)
    imageCaption = textAndImageSlide.shapes.add_textbox(captionLeft, captionTop, width=captionWidth, height=Inches(2))
    imageCaptionParagraph = imageCaption.text_frame.paragraphs[0]
    imageCaptionParagraph.alignment = PP_ALIGN.CENTER
    imageCaptionParagraph.line_spacing = Pt(1)
    imageCaptionRun = imageCaptionParagraph.add_run()
    imageCaptionRun.text = image['caption']
    imageCaptionFont = imageCaptionRun.font
    imageCaptionFont.name = 'Arial'
    imageCaptionFont.size = Pt(15)
            
    # добавляем нумерацию слайда
    pageNumberTextbox = textAndImageSlide.shapes.add_textbox(slideWidth - Inches(0.5), slideHeight - Inches(0.5), Inches(0.5), Inches(0.5))
    pageNumberParagraph = pageNumberTextbox.text_frame.paragraphs[0]
    pageNumberParagraph.alignment = PP_ALIGN.CENTER
    pageNumberRun = pageNumberParagraph.add_run()
    pageNumberRun.text = pageNumber
    pageNumberFont = pageNumberRun.font
    pageNumberFont.name = 'Arial'
    pageNumberFont.size = Pt(15)

    # удаление пустых плейсхолдеров
    for placeholder in textAndImageSlide.shapes.placeholders:
        if placeholder.has_text_frame and placeholder.text_frame.text == "":
            sp = textAndImageSlide.placeholders[1]._sp
            sp.getparent().remove(sp)

### Собираем слайд с одним предложением и одним изобржением.

In [ ]:
from pptx.util import Pt, Inches
from pptx.enum.text import PP_ALIGN

def addSentenceAndImageSlide(header, content, image, presentation, pageNumber):
    # слайд с текстом и изображением
    textAndImageSlideLayout = presentation.slide_layouts[1]
    textAndImageSlide = presentation.slides.add_slide(textAndImageSlideLayout)
    title = textAndImageSlide.shapes.title
    body = textAndImageSlide.shapes.placeholders[1]
    
    # двигаем текст повыше
    bodyLeft = body.left
    bodyTop = body.top - Inches(0.3)
    bodyWidth = body.width
    bodyHeight = body.height
    body.left, body.top, body.width, body.height = bodyLeft, bodyTop, bodyWidth, bodyHeight
    
    # заголовок текстового слайда
    titleParagraph = title.text_frame.paragraphs[0]
    titleParagraph.line_spacing = Pt(1)
    titleParagraph.alignment = PP_ALIGN.LEFT
    titleRun = titleParagraph.add_run()
    titleRun.text = header
    titleFont = titleRun.font
    titleFont.name = 'Arial Black'
    titleFont.size = Pt(28)
    
    # вставляем изображение в слайд
    heightToWidthRatio = image['height'] / image['width']
    linesSpace = ceil(len(content[0]['sentence']) / 70) * Inches(0.35) # сколько примерно места займет текст
    # изображение будет внизу
    left = Inches(0.5)
    top = body.top + linesSpace + Inches(0.2)
    width =  slideWidth - Inches(0.5) - Inches(0.5)
    height = int(heightToWidthRatio * width)
    if (slideHeight - top - height) < Inches(0.7):
        height = int(slideHeight - top - Inches(0.7))
        width =  int(height / heightToWidthRatio)
        left = int((slideWidth - width) / 2)
    # это нужно, чтобы сделать текстовый блок нормального размера
    bodyLeft = body.left
    bodyTop = body.top
    bodyWidth = body.width
    bodyHeight = top - body.top - Inches(0.2)
#     print('1', body.left, body.top, body.width, body.height)
    body.left, body.top, body.width, body.height = bodyLeft, bodyTop, bodyWidth, bodyHeight
#     print('1', body.left, body.top, body.width, body.height)
    # добавляем изображение
    slideImage = textAndImageSlide.shapes.add_picture(image['filePath'], left, top, width=width, height=height)
    # это нужно для того, чтобы потом вставить подпись в правильное место
    captionTop = top + heightToWidthRatio * width + Inches(0.2)
    captionLeft = left
    captionWidth = slideImage.width
        
    # вставляем подпись к изображению в слайд
    imageCaption = textAndImageSlide.shapes.add_textbox(left, captionTop, width=captionWidth, height=Inches(2))
    imageCaptionParagraph = imageCaption.text_frame.paragraphs[0]
    imageCaptionParagraph.alignment = PP_ALIGN.CENTER
    imageCaptionParagraph.line_spacing = Pt(1)
    imageCaptionRun = imageCaptionParagraph.add_run()
    imageCaptionRun.text = image['caption']
    imageCaptionFont = imageCaptionRun.font
    imageCaptionFont.name = 'Arial'
    imageCaptionFont.size = Pt(15)
        
    # содержимое текстового блока
    preprocessText(content)
    
    sentenceParagraph = body.text_frame.paragraphs[0]
    sentenceParagraph.alignment = PP_ALIGN.LEFT
    sentenceParagraph.line_spacing = Pt(1)
    sentenceParagraph.space_after = Pt(5)
    sentenceParagraphRun = sentenceParagraph.add_run()
    sentenceParagraphFont = sentenceParagraphRun.font
    sentenceParagraphFont.name = 'Arial'
    sentenceParagraphFont.size = Pt(18)
    sentenceParagraphRun.text = content[0]['sentence']
    sentenceParagraph._pPr.insert(0, etree.Element("{http://schemas.openxmlformats.org/drawingml/2006/main}buNone"))

    # добавляем нумерацию слайда
    pageNumberTextbox = textAndImageSlide.shapes.add_textbox(slideWidth - Inches(0.5), slideHeight - Inches(0.5), Inches(0.5), Inches(0.5))
    pageNumberParagraph = pageNumberTextbox.text_frame.paragraphs[0]
    pageNumberParagraph.alignment = PP_ALIGN.CENTER
    pageNumberRun = pageNumberParagraph.add_run()
    pageNumberRun.text = pageNumber
    pageNumberFont = pageNumberRun.font
    pageNumberFont.name = 'Arial'
    pageNumberFont.size = Pt(15)
    
    # удаление пустых плейсхолдеров
    for placeholder in textAndImageSlide.shapes.placeholders:
        if placeholder.has_text_frame and placeholder.text_frame.text == "":
            sp = textAndImageSlide.placeholders[1]._sp
            sp.getparent().remove(sp)

### Собираем слайд «Спасибо за внимание!»

In [ ]:
def addThanksSlide(presentation, pageNumber):
    # слайд "спасибо за внимание"
    thankSlideLayout = presentation.slide_layouts[0]
    thankSlide = presentation.slides.add_slide(thankSlideLayout)
    thank = thankSlide.shapes.title

    # текст "Спасибо за внимание!"
    thankParagraph = thank.text_frame.paragraphs[0]
    thankParagraph.alignment = PP_ALIGN.LEFT
    thankRun = thankParagraph.add_run()
    thankRun.text = "Спасибо за внимание!"
    thankFont = thankRun.font
    thankFont.name = 'Arial Black'
    thankFont.size = Pt(28)
    
    # добавляем нумерацию слайда
    pageNumberTextbox = thankSlide.shapes.add_textbox(slideWidth - Inches(0.5), slideHeight - Inches(0.5), Inches(0.5), Inches(0.5))
    pageNumberParagraph = pageNumberTextbox.text_frame.paragraphs[0]
    pageNumberParagraph.alignment = PP_ALIGN.CENTER
    pageNumberRun = pageNumberParagraph.add_run()
    pageNumberRun.text = pageNumber
    pageNumberFont = pageNumberRun.font
    pageNumberFont.name = 'Arial'
    pageNumberFont.size = Pt(15)

    # удаление пустых плейсхолдеров
    for placeholder in thankSlide.shapes.placeholders:
        if placeholder.has_text_frame and placeholder.text_frame.text == "":
            sp = thankSlide.placeholders[1]._sp
            sp.getparent().remove(sp)

### Собираем презентацию.

In [ ]:
from pptx import Presentation
from math import ceil

presentation = Presentation()
slideHeight = presentation.slide_height
slideWidth = presentation.slide_width

In [ ]:
addTitleSlide(titlePageDict, presentation)

pageNumber = 2
for modelPart in presentationModel:
    if modelPart['content'] and modelPart['image']:
        if len(modelPart['content']) > 1:
            addTextAndImageSlide(modelPart['header'], modelPart['content'], modelPart['image'], presentation, str(pageNumber))
            pageNumber += 1
        if len(modelPart['content']) == 1:
            addSentenceAndImageSlide(modelPart['header'], modelPart['content'], modelPart['image'], presentation, str(pageNumber))
            pageNumber += 1
    if not modelPart['content'] and modelPart['image']:
        addImageSlide(modelPart['header'], modelPart['image'], presentation, str(pageNumber))
        pageNumber += 1
    if modelPart['content'] and not modelPart['image']:
        addTextSlide(modelPart['header'], modelPart['content'], presentation, str(pageNumber))
        pageNumber += 1
        
addThanksSlide(presentation, str(pageNumber))

### Сохраняем презентацию в формате pptx.

In [ ]:
presentation.save(presentationFile)

## Оценка содержимого презентации.

### Собираем содержимое презентации в единый текст и считаем индекс удобочитаемости Флеша.

In [ ]:
import rusyllab
from pyrutok import Token, Sentence, Tokenizer, GraphemTag, TokenType
from pymorphy2 import MorphAnalyzer

def countСharacteristics(sentenceText):
    syllablesCount = 0
    wordsCount = 0
    sentencesCount = 1
    for sentence in Tokenizer(sentenceText):
        for token in sentence:
            if GraphemTag.contains(token.get_graphem_tag(), GraphemTag.CYRILLIC):
                syllablesCount += len(rusyllab.split_word(token.get_escaped_data()))
                wordsCount += 1
    return sentencesCount, wordsCount, syllablesCount

def countFleschReadingEase(sentencesCount, wordsCount, syllablesCount):
    asl = wordsCount / sentencesCount
    asw = syllablesCount / wordsCount
    return 206.835 - 1.3 * asl - 60.1 * asw

syllablesCount = 0
wordsCount = 0
sentencesCount = 0
presentationTextContent = ''

for modelPart in presentationModel:
    for sentence in modelPart['content']:
        sentenceText = sentence['sentence']
        if sentence['numberedListPart'] or sentence['bulletListPart']:
            if sentenceText.split(' ', 1)[0] in numbers or \
               sentenceText.split(' ', 1)[0] in listMarkers:
                sentenceText = sentenceText.split(' ', 1)[1]
        delimeter = ' ' if presentationTextContent else ''
        presentationTextContent += delimeter + sentenceText
        sentencesCountPart, wordsCountPart, syllablesCountPart = countСharacteristics(sentenceText)
        sentencesCount += sentencesCountPart
        wordsCount += wordsCountPart
        syllablesCount += syllablesCountPart
    if modelPart['image']:
        sentenceText = ''
        delimeter = ' ' if presentationTextContent else ''
        if isinstance(modelPart['image'], list):
            sentenceText = modelPart['image'][0]['caption']
            presentationTextContent += delimeter + sentenceText
        else:
            sentenceText = modelPart['image']['caption']
            presentationTextContent += delimeter + sentenceText
        sentencesCountPart, wordsCountPart, syllablesCountPart = countСharacteristics(sentenceText)
        sentencesCount += sentencesCountPart
        wordsCount += wordsCountPart
        syllablesCount += syllablesCountPart
        
fleschReadingEase = countFleschReadingEase(sentencesCount, wordsCount, syllablesCount)

print(fleschReadingEase)

### Оценка с помощью метрики ROUGE.

In [ ]:
from sumeval.metrics.rouge import RougeCalculator

contentToCompare = ''

rouge = RougeCalculator(stopwords=True, lang="ru")

rouge1 = rouge.rouge_n(
    summary=presentationTextContent,
    references=contentToCompare,
    n=1
)

rouge2 = rouge.rouge_n(
    summary=presentationTextContent,
    references=contentToCompare,
    n=2
)

rouge3 = rouge.rouge_n(
    summary=presentationTextContent,
    references=contentToCompare,
    n=3
)

rougeL = rouge.rouge_l(
    summary=presentationTextContent,
    references=contentToCompare
)

print("ROUGE-1: {}, ROUGE-2: {}, ROUGE-3: {}, ROUGE-L: {}".format(
    rouge1, rouge2, rouge3, rougeL
).replace(", ", "\n"))